In [31]:
# %%
%load_ext autoreload
%autoreload 2
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import pandas as pd
import seaborn as sns
import math
from collections import Counter
from functools import reduce
#For interactive plots 

import plotly.graph_objects as go #pip install plotly
import dash
from dash import dcc, html #pip install dash
from dash.dependencies import Input, Output #pip install dash-bootstrap-components

#load the statistical libraries
from statsmodels.stats import diagnostic
from scipy import stats

from src.data.utils import get_franchise_movies, get_franchise_data, get_inflation_rate, get_movie, get_genre_counts


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [32]:
#loading the inflation rate data set from the columbia bank 
inflation_rate_df=get_inflation_rate()

#loading the complete data set 
movies_df_raw = pd.read_csv('data/movie_metadata_with_tmdb.csv')

#loading the data set for the franchise movies 
franchise_df=get_franchise_movies(movies_df_raw,inflation_rate_df,'data/missing_dates_manualsearch.csv')

#Data frame with all the movies with processing
movies_df=get_movie(movies_df_raw,inflation_rate_df)

#Data frame with movies that are not part of a franchise
movies_no_franchise_df = movies_df[movies_df['collection_id'].isna()].reset_index(drop=True)

#Data frame of only the franchise name and their characteristics
franchise_data_df = get_franchise_data(franchise_df)

# Drop franchises where any movie in the franchise does not have a release date
franchise_df = franchise_df.groupby('collection_id').filter(lambda x: x['Movie release date'].notna().all())
#print("Number of movies with all movies having release dates : ", len(franchise_df_2))


In [33]:
from sklearn.feature_extraction.text import CountVectorizer
def get_1_2_movies(data): 
    """
    Function to get the 1_st and 2_nd movie franhcise to analyse it in the tree and KNN model
    """
    # Drop unnecessary columns
    data.drop(columns=[
    'Freebase movie ID', 'Movie release date', 'Movie languages (Freebase ID:name tuples)', 
    'vote_count', 'tmdb_original_language', 'Movie release date corrected', 'box_office', 
    'Movie runtime', 'Movie countries (Freebase ID:name tuples)', 'CPI', 'budget', 'tmdb_id','profit'
    ], inplace=True)
    
    data['movie_order'] = data['movie_order'].astype(int)

    data = data[(data['movie_order'] == 1) | (data['movie_order'] == 2)]
    data = data.pivot(index='collection_id', columns='movie_order')
    data.columns = ['_'.join(map(str, col)).strip() for col in data.columns]
    data = data.drop(columns =['years_diff_bt_pre_movies_1','collection_size_2'], axis=1)

    #movie_genre = data['genres'].apply(lambda x: ', '.join(set([genre for sublist in x for genre in sublist])))

    return data 
    
    

In [34]:
franchise_tree = get_1_2_movies(franchise_df)

In [37]:

franchise_tree=franchise_tree.dropna()
franchise_tree
